# Notebook Importuri

In [3]:
import pandas as pd
import numpy as np

# Constantele

In [78]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-token.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500 #gmail sou hotmail folosec undeva la 10000-15000 de cuvinte 

# Citeste si Incarca Features din fisierele .txt in NumPy Array

In [5]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [6]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [7]:
sparse_train_data[:5]# primele 5 din capul listei

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [8]:
sparse_train_data[-5:] #doc id, word id, categoria si  

array([[5795, 2047,    0,    2],
       [5795, 2053,    0,    1],
       [5795, 2299,    0,    1],
       [5795, 2373,    0,    1],
       [5795, 2442,    0,    1]])

In [10]:
print('Nr of rows in training file', sparse_train_data.shape[0])
print('Nr of rows in test file', sparse_test_data.shape[0])

Nr of rows in training file 258328
Nr of rows in test file 117621


In [12]:
print('Nr of emails is training files', np.unique(sparse_train_data[:,0]).size)
print('Nr of emails is test files', np.unique(sparse_test_data[:,0]).size)

Nr of emails is training files 4015
Nr of emails is test files 1724


## Cum sa cream un Empty DataFrame

In [14]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [15]:
len(column_names)

2502

In [17]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [20]:
#creare DataFrame pentru training
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [21]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
sparse_train_data[10:13]

array([[ 0, 31,  1,  1],
       [ 0, 33,  1,  1],
       [ 0, 45,  1,  1]])

In [23]:
sparse_train_data[10][3]

1

# Crearea Full Matric pentru Sparse Matrix

In [24]:
def make_full_matrix( sparse_matrix, nr_words, doc_idx=0, word_idx= 1, cat_idx=2, freq_idx=3):
    """
    Dintr-o matrice completă dintr-o matrice rară. Întoarceți un cadru de date pandas.
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- dimensiunea vocabularului. Numarul total de tokens.
    doc_ids -- positia doc_id in sparse matrix. Ca default: 1 coloana
    word_idx -- positia word_id in sparse matrix. Ca default: 2 coloana
    cat_idx -- positia categorie/label(spam (1) si nonspam(0)) in sparse matrix. Ca default: 3 coloana
    freq_idx -- positia apariția cuvântului in sparse matrix. Ca default: 4 coloana
    """

    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)

    for i in range(sparse_matrix.shape[0]):
        # luam datelele din sparse matrix
        doc_nr =sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label =sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]

        #populam full matrix cu date in anumite celule
        #selectam o anumita celula din acest data frame, selectam a doua valoare din []
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr    # selectam 'DOC_ID'care se va dece in coloana de document id, doc_nr reprezinta randurile, doc_nr corespunde sparse_matrix, si ii setam valoarea
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence


    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix



In [25]:
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

In [26]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,13,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Training Naive Bayes Model
# Caluculeaza probabilitate de Spam

In [ ]:
# Challenge: Calculați probabilitatea de spam - procentul de mesaje spam din setul de date de instruire. Stocați această valoare într-o variabilă numită prob_spam

In [27]:
full_train_data.CATEGORY.size # nr total de mesaje in dataset

4015

In [28]:
full_train_data.CATEGORY.sum() #nr de mesaje spam 

1250

In [31]:
prob_spam = full_train_data.CATEGORY.sum()/ full_train_data.CATEGORY.size
print('Probabilitatea de a fi spam este: ', prob_spam)

Probabilitatea de a fi spam este:  0.31133250311332505


# Numarul Total de Cuvinte / Tokens

In [32]:
#cream un subset pentru full_train_data
full_train_feaatures = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_feaatures.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
#nr. total de tokens per email
email_lengths = full_train_feaatures.sum(axis=1)
email_lengths.shape

(4015,)

In [34]:
email_lengths[:5]

DOC_ID
0     87
1     53
2     40
3    181
4     43
dtype: int64

In [35]:
total_wc = email_lengths.sum() # nr toatal de token 
total_wc

430993

# Numarul Tokens in Spam Si Ham Emailuri

Challenge: 

Creați un subset din seria de e-mail_lengths care conține doar mesajele spam. Apelați subseturile spam_lengths. Apoi, numărați numărul total de cuvinte care apar în e-mailurile spam.

Faceți același lucru pentru e-mailurile non-spam. Creați un subset denumit ham_lengths. Apoi, numărați numărul total de cuvinte care apar în e-mailurile de ham.

In [36]:
spam_lengths = email_lengths[full_train_data.CATEGORY ==1]
spam_lengths.shape

(1250,)

In [37]:
spam_wc = spam_lengths.sum()# nr total de tokens(cuvinte) in emailuri cu spam
spam_wc


178651

In [38]:
ham_lengths = email_lengths[full_train_data.CATEGORY ==0]
ham_lengths.shape

(2765,)

In [41]:
nonspam_wc = ham_lengths.sum()# nr total de tokens(cuvinte) in emailuri cu nonspam
nonspam_wc

252342

In [42]:
email_lengths.shape[0] -spam_lengths.shape[0] -ham_lengths.shape[0]#verificare

0

In [43]:
spam_wc +nonspam_wc - total_wc #verificare

0

In [46]:
# Challenge: Vă puteți da seama dacă e-mailurile spam sau e-mail-urile non-spam tind să fie mai lungi? Ce categorie conține mai multe tokens? Faceți o bănuială și apoi verificați intuiția folosind setul de date
print('Average nr of words in spam emails {:.0f}' .format(spam_wc / spam_lengths.shape[0]))
print('Average nr of words in ham emails {:.3f}' .format(nonspam_wc / ham_lengths.shape[0]))

Average nr of words in spam emails 143
Average nr of words in ham emails 91.263


## Rezumarea Tokens care au loc in Spam

In [47]:
full_train_feaatures.shape

(4015, 2500)

In [48]:
train_spam_tokens = full_train_feaatures.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1890,2,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
train_spam_tokens.shape

(1250, 2500)

In [51]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1 # Laplace Smoothing
summed_spam_tokens.shape

(2500,)

In [52]:
summed_spam_tokens.tail()

2495    11
2496    26
2497     4
2498    25
2499    28
dtype: int64

# Rezumarea Tokens care au loc in Ham

In [ ]:
#Challenge: Repetați acest proces pentru mesajele ham. Sumați tokens care apar în mesajele non-spam. Stocați valorile într-o variabilă numită summed_ham_tokens.

In [61]:
train_ham_tokens = full_train_feaatures.loc[full_train_data.CATEGORY == 0]
train_ham_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1896,1,2,1,0,1,1,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1898,2,1,0,1,1,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1899,2,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1900,3,4,0,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1901,2,2,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
train_ham_tokens.shape

(2765, 2500)

In [63]:
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1 # Laplace Smoothing
summed_ham_tokens.shape

(2500,)

In [64]:
summed_ham_tokens.tail()

2495    21
2496     9
2497    30
2498     3
2499     5
dtype: int64

In [65]:
train_ham_tokens[2499].sum() +1

5

# P(Token | Spam) - Probabilitatea ca un Token să apară dat de e-mail este spam

In [66]:
prob_tokens_spam =summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.012018
1    0.005167
2    0.006735
3    0.011184
4    0.006685
dtype: float64

In [67]:
prob_tokens_spam.sum()

1.0

# P(Token | Ham) - Probabilitatea ca un Token să apară dat de e-mail este nonspam

In [75]:
prob_tokens_nonspam =summed_ham_tokens / (nonspam_wc + VOCAB_SIZE) 

prob_tokens_nonspam[:5]

0    0.021515
1    0.010155
2    0.008021
3    0.003681
4    0.006322
dtype: float64

In [76]:
prob_tokens_nonspam.sum()

1.0

# P(Token) - Probabilitatea să apară tokenul

In [72]:
prob_tokens_all = full_train_feaatures.sum(axis=0) / total_wc

In [73]:
prob_tokens_all.sum()

1.0

# Salveaza Modelul de Trained

In [77]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# # Challenge: Creați o matrice completă din sparse_test_data. Timpul apelului funcțional. Cât timp îi ia? Separați caracteristicile și valorile țintă. Salvați aceste fișiere .txt separate: un fișier TEST_TARGET_FILE și un fișier TEST_FEATURE_MATRIX.

In [79]:
sparse_test_data.shape

(117621, 4)

In [80]:
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

In [81]:
x_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [82]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, x_test)